In [9]:
import os 
import sys
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import torch 
import nltk
import spacy
import tensorflow as tf

In [10]:
import platform
platform.platform()

torch.backends.mps.is_built()

if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [11]:

#Importing Configs
# Define the path where config.py is located
#Mac
os.chdir('/Users/kylenabors/Documents/GitHub/Finance-ML-Modeling')
#Linux
#os.chdir('/home/kwnabors/Documents/GitHub/Finance-ML-Modeling')
config_file_path = os.getcwd()
print(config_file_path)

# Add this path to the sys.path
sys.path.append(config_file_path)

import config
#Variables, Paramaters, and Pathnames needed for this script
database_file = config.database
database_folder = config.database_folder
bert_models = config.bert_models
bert_models_local = config.bert_models_local
keywords = config.keywords
finbert_models = config.finbert_models

Body = config.Body
Model = config.Model
Model_Subfolder = f'/{Body} Texts/{Model}'
Model_Folder = config.texts
Model_Folder = Model_Folder + Model_Subfolder

df = pd.read_csv(f"{Model_Folder}/{Model}_texts_long.csv")  
df = df[df['language'] == 'en']
df_2 = pd.read_csv(f"{Model_Folder}/{Model}_texts.csv")  
df_2 = df_2[df_2['language'] == 'en']


/Users/kylenabors/Documents/GitHub/Finance-ML-Modeling


In [12]:
#Finbert 
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

tokenizer_1 = AutoTokenizer.from_pretrained("ProsusAI/finbert", force_download=True)
model_1 = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert", force_download=True)

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',
                                                        num_labels=3, 
                                                        force_download=True)

finbert = finbert.to('mps')

tokenizer_2 = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone', force_download=True)
nlp = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer_2)

labels = {0:'neutral', 1:'positive',2:'negative'}
out_1= []
out_2 = []
sent_val = list()
tone_val = list()
long = 0
errors = 0

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [13]:

for index, row in df_2.iterrows():
    docs = row["segment"]
    timestamps = row['date']
    type = row['type']
    title = row['title']
    docs = str(docs)
    doc_num = row['doc_num']
    
    try:
        inputs_1 = tokenizer_2(docs, padding=True, return_tensors="pt").to('mps')
        outputs = finbert(**inputs_1)[0]
        outputs = labels[np.argmax(outputs.cpu().detach().numpy())]
        results = outputs
    except:
        errors += 1
        continue

    out_1.append([doc_num, timestamps, title, results, docs])
    
print(f'Errors: {errors}')

Errors: 192


In [14]:
df_out_1 = pd.DataFrame(out_1, columns=["doc_num", "date", "title", "sentiment", "segment"])

print(df_out_1.head())

df_out_1["sentiment"] = df_out_1["sentiment"].replace({'positive': 1, 'neutral' : 0, 'negative' : -1})

df_out_1_2 = df_out_1[["doc_num", "sentiment"]]
df_out_1_2 = df_out_1_2.groupby(['doc_num']).mean()

print(df_out_1.head())

print(f'The analysis failed {errors} times.')


  doc_num        date    title sentiment  \
0    Fed0  1998-07-01  minutes   neutral   
1    Fed0  1998-07-01  minutes   neutral   
2    Fed0  1998-07-01  minutes   neutral   
3    Fed0  1998-07-01  minutes   neutral   
4    Fed0  1998-07-01  minutes  negative   

                                             segment  
0  the manager of the system open market account ...  
1  the manager also reported on developments in d...  
2  the committee then turned to a discussion of t...  
3  a summary of the economic and financial inform...  
4  the information reviewed at this meeting sugge...  
  doc_num        date    title  sentiment  \
0    Fed0  1998-07-01  minutes          0   
1    Fed0  1998-07-01  minutes          0   
2    Fed0  1998-07-01  minutes          0   
3    Fed0  1998-07-01  minutes          0   
4    Fed0  1998-07-01  minutes         -1   

                                             segment  
0  the manager of the system open market account ...  
1  the manager also repo

In [15]:
for index, row in df.iterrows():
    docs = row["segment"]
    timestamps = row['date']
    #type = row['type']
    title = row['title']
    docs = str(docs)
    doc_num = row['doc_num']
    
    inputs_2 = tokenizer_2(docs, return_tensors="pt", padding=True, truncation=True, max_length=511).to('mps')
    outputs_2 = finbert(**inputs_2)[0]
    val_2 = labels[np.argmax(outputs_2.cpu().detach().numpy())]
    #tone_val.append(val_2)

    out_2.append([doc_num, timestamps, title, type, docs, val_2])



In [16]:
df_out_2 = pd.DataFrame(out_2, columns=["doc_num", "date", "title", "type", "segment", "tone"])


df_out = df_out_2.merge(df_out_1_2, on='doc_num', how='inner')

df_out.to_csv(f"{finbert_models}/{Body}_{Model}_finbert model.csv")  
df_out_1.to_csv(f"{finbert_models}/{Body}_{Model}_finbert model_line.csv")  

print('done')

done
